In [17]:
import numpy as np

In [18]:
import pandas as pd 
import pybedtools
from pybedtools import BedTool
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [19]:
data=pd.read_csv("/home/user/data2/lit/project/ZNF271/02-APA/annotation/terminal_exon/gencode.v41.annotation.terminal_exon.genePredExt",sep='\t',header=None)
data.columns=["chr","strand","start","end","gene_id"]
data.head()

,chr,strand,start,end,gene_id
0,chr1,-,100000636,100000739,ENSG00000202259.1
1,chr8,-,100002321,100002364,ENSG00000132554.21
2,chr11,+,9998685,10000290,ENSG00000286561.1
3,chr11,+,100003801,100003968,ENSG00000149972.12
4,chr8,+,100004582,100004649,ENSG00000212414.1


In [20]:
pa_anno=pd.read_csv("/home/user/data2/lit/project/ZNF271/02-APA/annotation/terminal_exon/ENCFF678TEN.PAusage.bed6+",sep='\t',header=None)
pa_anno.columns=["chr","start","end","gene_id","cnt","stand","pa_usage"]
pa_anno=pa_anno[pa_anno.cnt>1]

In [21]:
data_reverse=data[data.strand=='-']
data_forward=data[data.strand=='+']
data_reverse.head()

,chr,strand,start,end,gene_id
0,chr1,-,100000636,100000739,ENSG00000202259.1
1,chr8,-,100002321,100002364,ENSG00000132554.21
5,chr2,-,100006148,100007017,ENSG00000144218.21
7,chr2,-,100007151,100007460,ENSG00000144218.21
8,chr2,-,100007212,100007460,ENSG00000144218.21


In [22]:
tmp = [data_reverse, data_forward]
for i in range(len(tmp)):
    if i == 0:
        col = 'end' 
    else:
        col = 'start'
        
    cnt = pd.DataFrame(tmp[i].gene_id.value_counts()).reset_index()
    cnt.columns = ['gene_id','cnt1']
    tmp[i] = pd.merge(tmp[i], cnt, on='gene_id',how='left')

    cnt = pd.DataFrame(tmp[i][['gene_id', col]].value_counts()).reset_index()
    cnt.columns = ['gene_id',col,'cnt2']
    tmp[i] = pd.merge(tmp[i], cnt, on=['gene_id',col],how='left')
    
    tmp[i]['freq'] = tmp[i].cnt2 / tmp[i].cnt1
    tmp[i] = tmp[i][(tmp[i]['freq'] > 0.5) & (tmp[i]['cnt2'] > 1)]
    
    if i == 0:
        tmp[i] = tmp[i].groupby(['gene_id','chr','strand']).agg({"start":["max","max","min"], "end":"max"}).reset_index()
        tmp[i].columns = ['gene_id','chr','strand',"proximal_start","distal_end","distal_start",'proximal_end']
        tmp[i] = tmp[i][['gene_id','chr','strand',"proximal_start","proximal_end","distal_start",'distal_end']]
        # tmp[i] = tmp[i][(tmp[i].proximal_end-tmp[i].proximal_start >= 100) & (tmp[i].distal_end-tmp[i].distal_start >= 100)]
        
    else:
        tmp[i] = tmp[i].groupby(['gene_id','chr','strand']).agg({"start":["min"], "end":["min","min","max"]}).reset_index()
        tmp[i].columns = ['gene_id','chr','strand',"proximal_start","proximal_end","distal_start",'distal_end']
        tmp[i] = tmp[i][['gene_id','chr','strand',"proximal_start","proximal_end","distal_start",'distal_end']]
        # tmp[i] = tmp[i][(tmp[i].proximal_end-tmp[i].proximal_start >= 100) & (tmp[i].distal_end-tmp[i].distal_start >= 100)]
    
    
        

/tmp/ipykernel_49852/3259046306.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  tmp[i] = tmp[i].groupby(['gene_id','chr','strand']).agg({"start":["max","max","min"], "end":"max"}).reset_index()
/tmp/ipykernel_49852/3259046306.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  tmp[i] = tmp[i].groupby(['gene_id','chr','strand']).agg({"start":["max","max","min"], "end":"max"}).reset_index()
/tmp/ipykernel_49852/3259046306.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
  tmp[i] = tmp[i].groupby(['gene_id','chr','strand']).agg({"start":["min"], "end":["min","min","max"]}).reset_index()
/tmp/ipykernel_49852/3259046306.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
  tmp[i] = tmp[i].groupby(['gene_id','chr','strand']).agg({"start":["min"], "end":["min","min","max"]}).reset_index()


In [23]:
tmp_concat=pd.concat([tmp[1],tmp[0]],axis=0)
tmp_concat

,gene_id,chr,strand,proximal_start,proximal_end,distal_start,distal_end
0,ENSG00000000460.17,chr1,+,169852789,169853037,169853037,169854080
1,ENSG00000001167.15,chr6,+,41097356,41097950,41097950,41102403
2,ENSG00000002726.21,chr7,+,150860942,150861219,150861219,150861504
3,ENSG00000002745.13,chr7,+,121338880,121339458,121339458,121341104
4,ENSG00000002919.15,chr17,+,48121234,48121288,48121288,48123601
...,...,...,...,...,...,...,...
5715,ENSG00000289870.1,chrX,-,109734671,109735046,109734671,109734671
5716,ENSG00000289873.1,chr22,-,32265957,32266352,32265954,32265957
5717,ENSG00000289881.1,chr1,-,84614069,84614667,84614067,84614069
5718,ENSG00000289950.1,chr19,-,50490872,50491100,50490872,50490872


In [24]:
encode_anno_1=pd.DataFrame(data={
    'chr':tmp[1]['chr'],
    'start':tmp[1]['proximal_start'],
    'end':tmp[1]['distal_end']+30,
    'name':tmp[1]['gene_id'],
    'score':".",
    'strand':tmp[1]['strand']
})

encode_anno_0=pd.DataFrame(data={
    'chr':tmp[0]['chr'],
    'start':tmp[0]['distal_start']-30,
    'end':tmp[0]['proximal_end'],
    'name':tmp[0]['gene_id'],
    'score':".",
    'strand':tmp[0]['strand']
})


encode_anno=pd.concat([encode_anno_0,encode_anno_1],axis=0)

In [25]:
encode_bed=BedTool.from_dataframe(encode_anno)
iso_bed=BedTool.from_dataframe(pa_anno)
len(encode_bed)
# intersect
iso_and_encode=iso_bed.intersect(encode_bed,wa=True,wb=True)
iso_and_encode_df=iso_and_encode.to_dataframe(header=None)
# some genes may overlap
iso_and_encode_df=iso_and_encode_df[iso_and_encode_df.iloc[:,3]==iso_and_encode_df.iloc[:,10]]
# subset columns
iso_and_encode_df=iso_and_encode_df.iloc[:,[0,1,2,3,5]]
iso_and_encode_df.columns=['chr','start','end','gene_id',"strand"]
len(iso_and_encode_df)

cnt=pd.DataFrame(iso_and_encode_df.iloc[:,[3]].value_counts()).reset_index()
len(cnt[cnt.iloc[:,1]>1])
         
m=pd.merge(iso_and_encode_df, cnt[cnt.iloc[:,1]>1], on='gene_id',how='right')

m.head()

m_g=m.groupby(['gene_id']).agg({"end":["min","max"]}).reset_index()
len(m_g)

11662

/home/user/BGM/lit/.local/lib/python3.9/site-packages/pybedtools/bedtool.py:3682: UserWarning: Default names for filetype bed are:
['chrom', 'start', 'end', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts']
but file has 13 fields; you can supply custom names with the `names` kwarg
  warn(


10012

2227

,chr,start,end,gene_id,strand,0
0,chr8,48060937,48060938,ENSG00000169139.12,+,16
1,chr8,48061387,48061388,ENSG00000169139.12,+,16
2,chr8,48061482,48061483,ENSG00000169139.12,+,16
3,chr8,48061567,48061568,ENSG00000169139.12,+,16
4,chr8,48061676,48061677,ENSG00000169139.12,+,16


2227

In [26]:
pd.merge(m_g,tmp[1],on='gene_id',how='inner')
m_g_1=pd.merge(m_g,tmp[1],on='gene_id',how='inner').iloc[:,[0,4,5,6,2,2,3]]
m_g_1.columns = ['gene_id','chr','strand',"proximal_start","proximal_end","distal_start",'distal_end']
m_g_1
pd.merge(m_g,tmp[0],on='gene_id',how='inner')
m_g_0=pd.merge(m_g,tmp[0],on='gene_id',how='inner').iloc[:,[0,4,5,3,7,2,3]]
m_g_0.columns = ['gene_id','chr','strand',"proximal_start","proximal_end","distal_start",'distal_end']
m_g_0

/tmp/ipykernel_49852/1211071859.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  pd.merge(m_g,tmp[1],on='gene_id',how='inner')


,gene_id,"(gene_id, )","(end, min)","(end, max)",chr,strand,proximal_start,proximal_end,distal_start,distal_end
0,ENSG00000002919.15,ENSG00000002919.15,48122743,48123071,chr17,+,48121234,48121288,48121288,48123601
1,ENSG00000004961.15,ENSG00000004961.15,11121876,11123072,chrX,+,11121611,11121879,11121879,11123086
2,ENSG00000005022.6,ENSG00000005022.6,119471237,119471311,chrX,+,119470900,119470940,119470940,119471396
3,ENSG00000005486.17,ENSG00000005486.17,75888430,75888920,chr7,+,75887991,75888268,75888268,75888926
4,ENSG00000005513.10,ENSG00000005513.10,986032,986978,chr16,+,984700,986978,986978,986979
...,...,...,...,...,...,...,...,...,...,...
1128,ENSG00000286340.3,ENSG00000286340.3,79080420,79080532,chr6,+,79079436,79079472,79079472,79081371
1129,ENSG00000286585.3,ENSG00000286585.3,155863195,155863740,chr3,+,155862647,155863194,155863194,155863739
1130,ENSG00000286961.2,ENSG00000286961.2,22758232,22758525,chr5,+,22758040,22760148,22760148,22760592
1131,ENSG00000289047.2,ENSG00000289047.2,32903466,32903758,chr6,+,32900919,32902160,32902160,32903758


/tmp/ipykernel_49852/1211071859.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  m_g_1=pd.merge(m_g,tmp[1],on='gene_id',how='inner').iloc[:,[0,4,5,6,2,2,3]]


,gene_id,chr,strand,proximal_start,proximal_end,distal_start,distal_end
0,ENSG00000002919.15,chr17,+,48121234,48122743,48122743,48123071
1,ENSG00000004961.15,chrX,+,11121611,11121876,11121876,11123072
2,ENSG00000005022.6,chrX,+,119470900,119471237,119471237,119471311
3,ENSG00000005486.17,chr7,+,75887991,75888430,75888430,75888920
4,ENSG00000005513.10,chr16,+,984700,986032,986032,986978
...,...,...,...,...,...,...,...
1128,ENSG00000286340.3,chr6,+,79079436,79080420,79080420,79080532
1129,ENSG00000286585.3,chr3,+,155862647,155863195,155863195,155863740
1130,ENSG00000286961.2,chr5,+,22758040,22758232,22758232,22758525
1131,ENSG00000289047.2,chr6,+,32900919,32903466,32903466,32903758


/tmp/ipykernel_49852/1211071859.py:5: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  pd.merge(m_g,tmp[0],on='gene_id',how='inner')


,gene_id,"(gene_id, )","(end, min)","(end, max)",chr,strand,proximal_start,proximal_end,distal_start,distal_end
0,ENSG00000002587.10,ENSG00000002587.10,11398365,11398820,chr4,-,11399757,11400113,11393149,11399757
1,ENSG00000004142.12,ENSG00000004142.12,28346633,28348139,chr17,-,28347660,28348231,28346632,28347660
2,ENSG00000005175.10,ENSG00000005175.10,47662253,47663315,chr12,-,47663292,47663590,47661248,47663292
3,ENSG00000005194.15,ENSG00000005194.15,57428190,57428553,chr16,-,57429176,57429280,57428186,57429176
4,ENSG00000005243.10,ENSG00000005243.10,48026169,48026434,chr17,-,48026181,48026475,48026166,48026181
...,...,...,...,...,...,...,...,...,...,...
1089,ENSG00000276462.2,ENSG00000276462.2,41100791,41100988,chr9,-,41101041,41101294,41100792,41101041
1090,ENSG00000280832.3,ENSG00000280832.3,126340914,126341702,chr11,-,126341595,126342322,126340881,126341595
1091,ENSG00000282826.4,ENSG00000282826.4,28580634,28583027,chr20,-,28583028,28583110,28580632,28583028
1092,ENSG00000282881.3,ENSG00000282881.3,46532165,46532924,chr1,-,46532165,46533650,46532165,46532165


/tmp/ipykernel_49852/1211071859.py:6: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  m_g_0=pd.merge(m_g,tmp[0],on='gene_id',how='inner').iloc[:,[0,4,5,3,7,2,3]]


,gene_id,chr,strand,proximal_start,proximal_end,distal_start,distal_end
0,ENSG00000002587.10,chr4,-,11398820,11400113,11398365,11398820
1,ENSG00000004142.12,chr17,-,28348139,28348231,28346633,28348139
2,ENSG00000005175.10,chr12,-,47663315,47663590,47662253,47663315
3,ENSG00000005194.15,chr16,-,57428553,57429280,57428190,57428553
4,ENSG00000005243.10,chr17,-,48026434,48026475,48026169,48026434
...,...,...,...,...,...,...,...
1089,ENSG00000276462.2,chr9,-,41100988,41101294,41100791,41100988
1090,ENSG00000280832.3,chr11,-,126341702,126342322,126340914,126341702
1091,ENSG00000282826.4,chr20,-,28583027,28583110,28580634,28583027
1092,ENSG00000282881.3,chr1,-,46532924,46533650,46532165,46532924


In [27]:
m_g_m=pd.concat([m_g_1,m_g_0])

m_g_m = m_g_m[(m_g_m.proximal_end-m_g_m.proximal_start >= 100) & (m_g_m.distal_end-m_g_m.distal_start >= 100)]

m_g_m

,gene_id,chr,strand,proximal_start,proximal_end,distal_start,distal_end
0,ENSG00000002919.15,chr17,+,48121234,48122743,48122743,48123071
1,ENSG00000004961.15,chrX,+,11121611,11121876,11121876,11123072
3,ENSG00000005486.17,chr7,+,75887991,75888430,75888430,75888920
4,ENSG00000005513.10,chr16,+,984700,986032,986032,986978
6,ENSG00000006283.18,chr17,+,50626016,50627071,50627071,50627465
...,...,...,...,...,...,...,...
1088,ENSG00000271538.7,chr4,-,184507199,184507376,184506128,184507199
1089,ENSG00000276462.2,chr9,-,41100988,41101294,41100791,41100988
1090,ENSG00000280832.3,chr11,-,126341702,126342322,126340914,126341702
1092,ENSG00000282881.3,chr1,-,46532924,46533650,46532165,46532924


In [28]:
pd.concat([tmp[1],tmp[0]],axis=0).to_csv("/home/user/data2/lit/project/ZNF271/02-APA/annotation/terminal_exon/terminal_exon_annotation.txt",sep="\t",header=0,index=0)

In [29]:
m_g_m.to_csv("/home/user/data2/lit/project/ZNF271/02-APA/annotation/terminal_exon/terminal_exon_annotation.ref_based.txt",sep="\t",header=0,index=0)